In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zeeshanyounas001/email-spam-detection")

print("Path to dataset files:", path)

100%|████████████████████████████████████████████████████████████████████████████████| 418k/418k [00:01<00:00, 375kB/s]

Extracting files...
Path to dataset files: C:\Users\ASUS\.cache\kagglehub\datasets\zeeshanyounas001\email-spam-detection\versions\2


In [6]:
# =============================================
# 📧 Spam Email Detection - Complete Project
# =============================================

# 1. Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import string
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords (only first time)
nltk.download("stopwords")

# =============================================
# 2. Load Dataset
# =============================================
df = pd.read_csv("spam_mail.csv")
print("Dataset Loaded Successfully!")
print(df.head())

# =============================================
# 3. Clean Text Function
# =============================================

df.rename(columns={"Masseges": "Message"}, inplace=True)
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()                                         # lowercase
    text = re.sub(r'\d+', '', text)                             # remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    words = text.split()
    words = [word for word in words if word not in stop_words]  # remove stopwords
    return " ".join(words)

df["clean_message"] = df["Message"].apply(clean_text)

# =============================================
# 4. Encode Labels (ham=0, spam=1)
# =============================================
le = LabelEncoder()
df["label"] = le.fit_transform(df["Category"])

# =============================================
# 5. TF-IDF Vectorization
# =============================================
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df["clean_message"])
y = df["label"]

# =============================================
# 6. Train-Test Split
# =============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =============================================
# 7. Train Models
# =============================================

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

# Logistic Regression
log_model = LogisticRegression(max_iter=200)
log_model.fit(X_train, y_train)
log_pred = log_model.predict(X_test)

# =============================================
# 8. Evaluate Models
# =============================================
print("\n============================")
print("🔹 Naive Bayes Results")
print("============================")
print("Accuracy:", accuracy_score(y_test, nb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, nb_pred))
print("Classification Report:\n", classification_report(y_test, nb_pred))

print("\n============================")
print("🔹 Logistic Regression Results")
print("============================")
print("Accuracy:", accuracy_score(y_test, log_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, log_pred))
print("Classification Report:\n", classification_report(y_test, log_pred))

# =============================================
# 9. Function to Predict New Message
# =============================================
def classify_email(text):
    cleaned = clean_text(text)
    vectorized = tfidf.transform([cleaned])
    pred = nb_model.predict(vectorized)[0]
    return "Spam" if pred == 1 else "Ham (Not Spam)"

# Test the function
print("\n============================")
print("🔍 Testing Custom Inputs")
print("============================")
print("Congratulations! You won a free iPhone!!!")
print(classify_email("Congratulations! You won a free iPhone!!!"))
print("Hi, can we meet tomorrow?")
print(classify_email("Hi, can we meet tomorrow?"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dataset Loaded Successfully!
  Category                                           Masseges
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...

🔹 Naive Bayes Results
Accuracy: 0.9775784753363229
Confusion Matrix:
 [[965   0]
 [ 25 125]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115


🔹 Logistic Regression Results
Accuracy: 0.9488789237668162
Confusion Matrix:
 [[961   4]
 [ 53  97]]
Classification Report:
               precision    r